In [1]:
import pandas as pd
import re
from unidecode import unidecode

In [2]:
benefit_map = {
    'Assistência médica':                    ['medica', 'saude', 'medico'],
    'Assistência odontológica':              ['odonto', 'dentista'],
    'Assistência psicológia':                ['psicolog', 'saude mental'],
    'Auxílio academia':                      ['academia', 'gympass', 'gym', 'total pass'],
    'Auxílio combustível':                   ['combustivel'],
    'Auxílio creche':                        ['creche'],
    'Auxílio desenvolvimento':               ['desenvolvimento'],
    'Auxílio estacionamento':                ['estacionamento'],
    'Auxílio farmácia':                      ['farmacia', 'medicamento'],
    'Auxílio fretado':                       ['fretado'],
    'Auxílio home office':                   ['auxilio home', 'custo home'],
    'Bicicletário':                          ['bicleta'],
    'Bolsa auxílio':                         ['bolsa auxilio'],
    'Café da manhã':                         ['cafe da manha'],
    'Cesta básica':                          ['cesta basica'],
    'Cesta de natal':                        ['natal'],
    'Clube de vantagens':                    ['vantagens'],
    'Consignado':                            ['consignado'],
    'Convênio com empresas parceiras':       ['convenio'],
    'Cooperativa de crédito':                ['cooperativa de credito'],
    'Day off aniversário':                   ['aniversario'],
    'Desconto em produtos':                  ['desconto em produtos'],
    'Ginástica laboral':                     ['ginastica laboral'],
    'Horário flexível':                      ['horario flexivel', 'flexibilidade'],
    'Massoterapia':                          ['massoterapia', 'massagem'],
    'Participação nos Lucros ou Resultados': ['lucros', 'plr', 'ppr'],
    'Plano de Aquisição de Ações':           ['aquisicao de acoes'],
    'Previdência privada':                   ['previdencia'],
    'Programa de remuneração variável':      ['remuneracao variavel', 'bonificac', 'premiac'],
    'Programa de treinamentos':              ['treinamento', 'capacitacao'],
    'Refeitório':                            ['refeitorio'],
    'Restaurante interno':                   ['restaurante'],
    'Sala de Jogos':                         ['jogos'],
    'Seguro de vida':                        ['vida'],
    'Vale-alimentação':                      ['alimentacao', 'alimentação', 'alimentaçao', 'alimentacão', ' va ', ' va,'],
    'Vale-cultura':                          ['vale cultura', 'vale-cultura'],
    'Vale-refeição':                         ['refeicao', 'refeição', 'refeiçao', 'refeicão' 'ticket', ' vr ', ' vr,'],
    'Vale-transporte':                       ['transporte'],
}

In [3]:
def get_senioridade(df):
    mapa_senioridade = {
        'Júnior': ['Júnior', 'Junior', 'Jr', 'Estágio', 'Estagiário', 'Trainee', 'trainne', 'I'],
        'Pleno': ['Pleno', 'Plena', 'Pl', 'Jr/Pl', 'II'],
        'Sênior': ['Sênior', 'Senior', 'Sr', 'Pl/Sr', 'Tech Lead', 'Líder', 'Especialista', 'III', 'Coordenadora']
    }

    # Inicializa todas as linhas com 'Não Informado'
    df['senioridade'] = 'Não Informado'

    for index, row in df.iterrows():
        for senioridade, termos in mapa_senioridade.items():
            for termo in termos:
                if re.search(r'\b' + termo + r'\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                   re.search(r'\b' + termo + r'\b', row['descricao'], flags=re.IGNORECASE):
                    df.at[index, 'senioridade'] = senioridade
                    break
            if df.at[index, 'senioridade'] != 'Não Informado':
                break
    return df

In [4]:
def identificar_cargo(df):
    for index, row in df.iterrows():
        cargo = None

        if re.search(r'\bCientista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bCientista\b', row['descricao'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Cientista de Dados'

        elif re.search(r'\bAnalista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                re.search(r'\bAnalista\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Analista de Dados'

        elif re.search(r'\bEngenheir\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                re.search(r'\bEngenheir\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Engenheiro de Dados'

        else:
            cargo = 'Não identificado'

        df.at[index, 'posicao'] = cargo

    return df

In [5]:
def identificar_regime(df):
    for index, row in df.iterrows():
        regime = None

        if re.search(r'\bPJ\b', row['descricao'], flags=re.IGNORECASE):
            regime = 'PJ'
        else:
            regime = 'CLT'

        df.at[index, 'regime'] = regime

    return df

In [6]:
def verificar_tipo_trabalho(descricao, dicionario):
    descricao = descricao.lower()  # Converte a descrição para minúsculas para facilitar a verificação

    for tipo, palavras_chave in dicionario.items():
        for palavra in palavras_chave:
            if palavra.lower() in descricao:
                return tipo

    return 'Híbrido'  # Retorna 'Indefinido' se nenhum tipo for encontrado

In [7]:
def format_benefits_list(description: str) -> list[str] | None:
    """Extrai e padroniza os benefícios a partir da descrição da vaga.

    Args:
        description (str): Descrição da vaga.

    Returns:
        list[str] | None: Lista com os benefícios padronizados, ou None se não forem encontrados.

    Exemplos:
    ---
    >>> format_benefits_list('Oferecemos vale-alimentação e plano de saúde.')
    ['Vale-alimentação', 'Plano de saúde']
    """

    try:
        # Inicializa uma lista para armazenar os benefícios extraídos
        description = description.replace('\n', ' ')
        extracted_benefits = []

        # Converte toda a descrição para minúsculas para facilitar a correspondência
        description_lower = description.lower()

        # Percorre o mapa de benefícios para verificar a presença de cada benefício na descrição
        for benefit, synonyms in benefit_map.items():
            # Verifica se algum sinônimo do benefício está presente na descrição
            if benefit == 'Vale-alimentação':
                pass
            if any(synonym.lower() in description_lower for synonym in synonyms):
                extracted_benefits.append(benefit)

        # Retorna a lista de benefícios extraídos, removendo duplicatas e mantendo a ordem original
        if extracted_benefits:
            return list(dict.fromkeys(extracted_benefits))
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
def get_skills_list(description: str) -> list[str] | None:
    try:
        skills_str = description
        skills_str = re.sub(r'\s+', ' ', skills_str)
        skills_str = re.sub(r'(?<=[a-z]|\))([A-Z])', r';\1', skills_str)
        skills_list = re.findall(r'(?:(?<=[\:\;\.\•\*]).*?[A-Z].*?(?=[\.\;\•\*\)]))', skills_str, flags=re.I)
        skills_list = list(map(lambda skill: skill.strip(), skills_list))
        return skills_list
    except:
        return None

def format_skills_list(skills_list: list[str], skills_map: dict[str, list]) -> list[str] | None:
    try:
        skills_list_formatted = []
        skills_str = ', '.join(skills_list)
        skills_str = replace_specifics_substrings(skills_str)
        skills_str = unidecode(skills_str).lower()

        for skills_key, skills_matches in skills_map.items():
            match_look = any(True if skill_word in skills_str else False for skill_word in skills_matches)
            if match_look:
                skills_list_formatted.append(skills_key)
            else:
                continue

        if len(skills_list_formatted) == 0:
            return None

        return skills_list_formatted

    except:
        return None

def replace_specifics_substrings(benefits_str: str) -> str:

    va_vr_matches = re.findall(r'V[R|A]\b', benefits_str)

    va_vr_map = zip(['Vale refeição', 'Vale alimentação'], ['VR', 'VA'])

    for va_vr_full, va_vr_short in va_vr_map:

        if va_vr_short in va_vr_matches:
            benefits_str = benefits_str.replace(va_vr_short, va_vr_full)

    return benefits_str

def build_skills_map(data, macro:bool = False):
    dict_micro_tema = {}
    dict_macro_tema = {}

    for index, row in data[['micro','palavras_chave']].iterrows():
        dict_micro_tema[row['micro']] = row['palavras_chave'].split(', ')

    if macro:
        df_macro_temas = data[['macro','micro']].groupby('macro').agg(palavras_chave = ('micro',lambda x: ', '.join(x.str.lower()))).reset_index()
        for index, row in df_macro_temas.iterrows():
            dict_macro_tema[row['macro']] = row['palavras_chave'].split(', ')
        return dict_macro_tema, dict_micro_tema
    else:
        return dict_micro_tema

def get_skills_list(description: str) -> list[str] | None:
    '''
    Separa palavras que eventualmente podem estar juntas
    Na descrição, palavras podem estar juntas, por isso essa função separa palavras chave que possam estar unidas
    Ajuste de formatação dos dados
    '''
    try:
        # na gupy a descricao geralmente é dividida em 4 seções: 0 -> 'JOB DESCRIPTION';                 1 -> 'RESPONSIBILITIES AND ASSIGNMENTS';
        #                                                        2 -> 'REQUIREMENTS AND QUALIFICATIONS'; 3 -> 'ADDITIONAL INFORMATION'
        # a lista de beneficios geralmente esta contida na seção 'ADDITIONAL INFORMATION' (existem alguns casos que isso não é verdade)

        # divide a descricao em seções
        skills_str = description

        # substitui multiplos espaços em sequência por apenas um
        skills_str = re.sub(r'\s+', ' ', skills_str)

        # adiciona um ponto-virgula ou ')' onde uma letra maiuscula é antecida por uma minuscula, precisamos dessa condição porque
        # em muitos casos não existe um caracter utilizado para deixar clara separação entre os benefícios dentro da string; nessa
        # situação, os beneficios estarão colados uns aos outros e a unica forma de diferenciá-los são os caracteres maiusculos.
        skills_str = re.sub(r'(?<=[a-z]|\))([A-Z])', r';\1', skills_str)  # Exemplo:  Gympass(ilimitado)Vale-TransporteConvenio
                                                                            #        -> Gympass(ilimitado);Vale-Transporte;Convenio

        # cria uma lista com substrings contendo uma letra maiúscula precedida e antecedida por um entre: ':', ';', '.', '•', '*'
        # esses simbolos geralmente antecedem / procedem os elementos da lista de beneficios dentro da string.
        skills_list = re.findall(r'(?:(?<=[\:\;\.\•\*]).*?[A-Z].*?(?=[\.\;\•\*\)]))', skills_str, flags=re.I)   # Exemplos: '* Gympass;' -> 'Gympass'
                                                                                                                #             '• PLR.'     -> 'PLR'

        # remove espaços vazios no final e inicio de cada elemento da lista
        skills_list = list( map( lambda skill: skill.strip(), skills_list ) )

        return skills_list

    except:
        return None

In [9]:
tipo_trabalho = {
    'Híbrido': ['híbrido', 'horário flexível'],
    'Remoto': ['remoto'],
    'Presencial': ['presencial']
}

In [10]:
df = pd.read_csv('../data/data_raw/vagas_glassdoor_raw.csv')

df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Engenheiro de dados,NaN,Analista de Dados Pleno (Data Science).,Quod,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Engenheiro de dados,NaN,Cientista de Dados - Inteligência Artificial,"LBCA - Lee, Brock, Camargo Advogados",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009177642279,"A LBCA é um escritório full service, formado p...","['TensorFlow', 'Statistics', 'Computer science..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-28,NaN,2024-03-13,Cientista de dados,NaN,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,ClearSale,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009147683343,JOB DESCRIPTIONA Área de Modelagem da ClearSal...,"['Statistics', 'E-commerce', 'SQL', 'Statistic..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista de dados,NaN,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/cas-c...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-02,NaN,2024-03-13,Engenheiro de dados,NaN,CAS | Cientista de Dados - Foco em Crédito,Sicredi,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009055258960,Integrar os times de dados da Diretoria Execut...,"['Statistics', 'Cloud infrastructure', 'Relati..."


In [11]:
df = df.drop_duplicates(subset=['codigo_vaga'])

df = get_senioridade(df)
df = identificar_cargo(df)
df = identificar_regime(df)
df['senioridade'].replace('NaN', 'Não informado')
df['modalidade'] = df['modalidade'].replace('trabalho remoto', 'Remoto')
df['modalidade'] = df['modalidade'].fillna('Presencial')
df['estado'] = df['estado'].replace('Distrito Federal', 'DF')
df['modalidade'] = df['modalidade'].replace('Trabalho remoto', 'Remoto')
df.loc[df['modalidade'] == 'Remoto', 'cidade'] = 'Não informado'
df.loc[df['modalidade'] == 'Remoto', 'estado'] = 'Todo o Brasil'
df['modalidade'] = (df['descricao'].apply(lambda x: verificar_tipo_trabalho(x, tipo_trabalho)))
df['beneficios'] = df['descricao'].apply(format_benefits_list)

/tmp/ipykernel_802734/1671119655.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'CLT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'regime'] = regime


In [12]:
dict_skills = pd.read_excel('../data/dicionario-skills.xlsx', sheet_name='Habilidades')
skills_map_macro, skills_map_micro = build_skills_map(dict_skills, macro=True)

df['habilidades_micro'] = df['descricao'].apply(lambda descricao: format_skills_list(get_skills_list(descricao), skills_map_micro))
df['habilidades_macro'] = df['habilidades_micro'].apply(lambda habilidade: format_skills_list(habilidade, skills_map_macro))

In [13]:
dict_competencias = pd.read_excel('../data/dicionario-skills.xlsx', sheet_name='Competencias')
competencias_map = build_skills_map(dict_competencias, macro=False)
df['competencias'] = df['descricao'].apply(lambda descricao: format_skills_list(get_skills_list(descricao), competencias_map))


In [14]:
df.head(5).T

,0,1,2,3,4
site_da_vaga,Glassdoor,Glassdoor,Glassdoor,Glassdoor,Glassdoor
link_site,https://www.glassdoor.com.br/job-listing/anali...,https://www.glassdoor.com.br/job-listing/cient...,https://www.glassdoor.com.br/job-listing/cient...,https://www.glassdoor.com.br/job-listing/cient...,https://www.glassdoor.com.br/job-listing/cas-c...
link_origem,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...
data_publicacao,2024-03-12,2024-03-08,2024-02-28,2024-03-05,2024-03-02
data_expiracao,NaN,NaN,NaN,NaN,NaN
data_coleta,2024-03-13,2024-03-13,2024-03-13,2024-03-13,2024-03-13
posicao,Analista de Dados,Cientista de Dados,Cientista de Dados,Cientista de Dados,Cientista de Dados
senioridade,Pleno,Não Informado,Pleno,Não Informado,Não Informado
titulo_da_vaga,Analista de Dados Pleno (Data Science).,Cientista de Dados - Inteligência Artificial,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,Cientista de Dados - Vaga Afirmativa para Mulh...,CAS | Cientista de Dados - Foco em Crédito
nome_empresa,Quod,"LBCA - Lee, Brock, Camargo Advogados",ClearSale,"Vagas, empregos, oportunidades",Sicredi


In [15]:
df['modalidade'].value_counts()

modalidade
Híbrido       8296
Presencial    1033
Remoto         535
Name: count, dtype: int64

In [16]:
df.to_excel('../data/data_clean/vagas_glassdoor_clean.xlsx', index=False)

In [17]:
df2 = df[df['titulo_da_vaga'].str.contains("Dados")]

In [18]:
df2.to_excel('../data/data_clean/vagas_glassdoor_clean_reduzido.xlsx', index=False)

In [19]:
df[(df['descricao'].str.contains('flexível'))]

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,...,contrato,regime,pcd,codigo_vaga,descricao,skills,beneficios,habilidades_micro,habilidades_macro,competencias
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Analista de Dados,Pleno,Analista de Dados Pleno (Data Science).,Quod,...,NaN,CLT,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',...","[Assistência odontológica, Auxílio academia, A...","[Teradata, Estatística, Excel, Linguagem R, Py...","[Analytics, Pacote Office, Programação, SQL]",[Colaboração]
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista de Dados,Não Informado,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",...,NaN,CLT,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin...","[Assistência médica, Assistência odontológica,...","[GCP, Estatística, Machine Learning, SQL]","[Cloud, Machine Learning, SQL]","[Raciocínio lógico, Criatividade, Comunicação]"
7,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-06,NaN,2024-03-13,Cientista de Dados,Sênior,Cientista de Dados - Especialista,Blip,...,NaN,CLT,NaN,1009160819949,O time de Ciência de Dados desempenha um papel...,"['Statistics', 'Spanish', 'Machine learning', ...","[Assistência odontológica, Assistência psicoló...","[Estatística, Machine Learning, Python]","[Machine Learning, Programação]","[Raciocínio lógico, Comunicação, Liderança]"
15,Glassdoor,https://www.glassdoor.com.br/job-listing/pesso...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Cientista de Dados,Pleno,Pessoa Cientista de Dados Pleno [Otimização],A3Data,...,NaN,CLT,NaN,1009177642400,"A A3Data é uma consultoria de dados, pioneira ...","['TensorFlow', 'Statistics', 'Data modeling', ...","[Auxílio academia, Auxílio desenvolvimento, Ho...","[Azure, AWS, Estatística, Spark, Machine Learn...","[Cloud, Engenharia de Software, Machine Learni...","[Storytelling, Comunicação, Autonomia, Colabor..."
17,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-17,NaN,2024-03-13,Cientista de Dados,Júnior,Cientista de Dados,Brivia,...,NaN,CLT,NaN,1007126351372,Vaga para nosso BANCO DE TALENTOS\nProcuramos ...,"['Statistics', 'Branding', 'Business intellige...","[Assistência odontológica, Auxílio academia, A...","[Estatística, Scala, Linguagem R, SQL]","[Programação, SQL]","[Resolução de Problemas, Autonomia]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12108,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-04-18,NaN,2024-04-22,Cientista de Dados,Sênior,Cientista de Dados Sênior (Remoto),GeekHunter,...,NaN,CLT,NaN,1009243439041,PYTHON\nRemoto\nSenioridade\nSênior\n(Mais de ...,"['Statistics', 'Relational databases', 'Englis...","[Assistência odontológica, Seguro de vida, Val...","[Estatística, Machine Learning, Scala, Java, P...","[Machine Learning, Programação]",None
12118,Glassdoor,https://www.glassdoor.com.br/job-listing/pesso...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-04-16,NaN,2024-04-22,Cientista de Dados,Pleno,Pessoa Cientista de Dados Pleno/Sênior,A3Data,...,NaN,CLT,NaN,1009239732017,A A3Data é uma consultoria referência em #Dado...,"['TensorFlow', 'Statistics', 'Data modeling', ...","[Auxílio academia, Auxílio desenvolvimento, Ho...","[AWS, GCP, Docker, Git, Estatística, Spark, Ma...","[Cloud, Engenharia de Software, Machine Learni...","[Comunicação, Autonomia, Colaboração]"
12144,Glassdoor,https://www.glas

In [20]:
df[df['modalidade'] == 'Híbrido'][ 'descricao'].iloc[0]

'Trabalhar na área de dados na QUOD é empolgante e desafiador. Nosso time procura entregar aos clientes a melhor informação ou produto, garantindo o melhor nível de excelência das entregas. Fazemos isso empregando as melhores técnicas de manipulação de dados conhecidas. Assim, conquistamos e mantemos a confianças de nossos clientes, garantindo a melhor informação possível disponível. Nossas ferramentas são extremamente adequadas à manipulação de quantidade colossais de dados. Assim, a QUOD está em linha com as maiores empresas do mundo quando o assunto é dados. No momento, temos uma excelente oportunidade para quem quer fazer parte desse time como Analista de Dados, na BU de crédito PF\nResponsabilidades e atribuições\nO profissional será responsável por extrair, carregar, transformar e integrar dados em diferentes formatos, ambientes e latências, para formar base de dados com a melhor informação de clientes com os dados disponíveis nos bancos de dados disponíveis.\nRequisitos e qualif

In [21]:
i = 0
df['descricao'][i]

'Trabalhar na área de dados na QUOD é empolgante e desafiador. Nosso time procura entregar aos clientes a melhor informação ou produto, garantindo o melhor nível de excelência das entregas. Fazemos isso empregando as melhores técnicas de manipulação de dados conhecidas. Assim, conquistamos e mantemos a confianças de nossos clientes, garantindo a melhor informação possível disponível. Nossas ferramentas são extremamente adequadas à manipulação de quantidade colossais de dados. Assim, a QUOD está em linha com as maiores empresas do mundo quando o assunto é dados. No momento, temos uma excelente oportunidade para quem quer fazer parte desse time como Analista de Dados, na BU de crédito PF\nResponsabilidades e atribuições\nO profissional será responsável por extrair, carregar, transformar e integrar dados em diferentes formatos, ambientes e latências, para formar base de dados com a melhor informação de clientes com os dados disponíveis nos bancos de dados disponíveis.\nRequisitos e qualif

In [22]:
df['beneficios'][i]

['Assistência odontológica',
 'Auxílio academia',
 'Auxílio creche',
 'Auxílio desenvolvimento',
 'Auxílio estacionamento',
 'Participação nos Lucros ou Resultados',
 'Seguro de vida',
 'Vale-alimentação',
 'Vale-refeição']

In [25]:
df2.shape

(1349, 23)